- IMPORT THE LIBRARIES YOU WILL USE
------------------------------------------
You only need to import packages one time per notebook session. To keep your
notebook clean and organized you can handle all imports at the top of your file.
The following are included for example purposed, feel free to modify or delete 
anything in this section.

In [ ]:
import csv
import pandas as pd
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
import seaborn as sns
from google.cloud import bigquery
from google.oauth2 import service_account
from google.cloud.bigquery import magics

In [ ]:
from sklearn.ensemble import RandomForestRegressor, HistGradientBoostingRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score
from sklearn.model_selection import GridSearchCV

In [ ]:
get_ipython().run_cell_magic('capture', '', "%logstop\n%logstart -t -r -q ipython_command_log.py global\n\n#- IRONHACKS RESEARCH TRACKING CODE\n#----------------------------------\n# The following code is used to help our research team understand how you \n# our notebook environment. We do not collect any personal information with\n# the following code, it is used to measure when and how often you work on\n# your submission files.\n\nimport os\nfrom datetime import datetime\nimport IPython.core.history as history\n\nha = history.HistoryAccessor()\nha_tail = ha.get_tail(1)\nha_cmd = next(ha_tail)\nsession_id = str(ha_cmd[0])\ncommand_id = str(ha_cmd[1])\ntimestamp = datetime.utcnow().isoformat()\nhistory_line = ','.join([session_id, command_id, timestamp]) + '\\n'\nlogfile = open(os.environ['HOME']+'/ipython_session_log.csv', 'a')\nlogfile.write(history_line)\nlogfile.close()\n")

In [ ]:
get_ipython().run_cell_magic('capture', '', "\n#- INSTALL ADDITIONAL LIBRARIES IF REQUIRED\n#------------------------------------------\n# This is normally not required. The hub environment comes preinstaled with \n# many packages that you can already use without setup. In case there is some\n# other library you would like to use that isn't on the list you run this command\n# once to install them.  If it is already installed this command has no effect.\n\n!python3 -m pip install pandas\n")

CONFIGURE THE BIGQUERY SETTINGS

In [ ]:
BIGQUERY_PROJECT = 'ironhacks-data'
bigquery_client = bigquery.Client(project=BIGQUERY_PROJECT)

In [ ]:
query = """
SELECT 
a.*,
b.average_wage
FROM `ironhacks-data.ironhacks_competition.unemployment_data`a

JOIN `ironhacks-data.ironhacks_competition.wage_data` b 
ON a.uu_id = b.uu_id


"""

In [ ]:
# QUERY THE DATA ONCE
query_job = bigquery_client.query(query)
data = query_job.to_dataframe()
#data['timeperiod']= pd.to_datetime(data['timeperiod'])
data.head()

In [ ]:
data.drop_duplicates(inplace=True)

In [ ]:
get_ipython().system('pip install db-dtypes')

In [ ]:
data = data.fillna(0)

In [ ]:
data = data.fillna(0)

In [ ]:
data.describe()

In [ ]:
data.sum(numeric_only=True)

In [ ]:
data.columns

In [ ]:
plt.figure(figsize=(12,10))
cor = data.corr()
sns.heatmap(cor, annot=True, cmap=plt.cm.Reds)
plt.show()

In [ ]:
y = data.total_claims
x = data.drop(['total_claims', 'timeperiod'], axis = 1)

In [ ]:
uuid, label = data['uu_id'].factorize(sort=True)

In [ ]:
x['uu_id'] = uuid

In [ ]:
x['tract_name'] = x['tract_name'].factorize()[0]

In [ ]:
x['top_category_employer1'] = x['top_category_employer1'].factorize()[0]
x['top_category_employer2'] = x['top_category_employer2'].factorize()[0]
x['top_category_employer3'] = x['top_category_employer3'].factorize()[0]

In [ ]:
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=0)

In [ ]:
model = HistGradientBoostingRegressor(loss="squared_error")
model.fit(x_train, y_train,
        verbose=False)  # Change verbose to True if you want to see it train

In [ ]:
model = HistGradientBoostingRegressor(loss="squared_error")
model.fit(x_train, y_train)

In [ ]:
model.score(x_test, y_test)

In [ ]:
query = """
SELECT * FROM `ironhacks-data.ironhacks_competition.prediction_list` 
"""

In [ ]:
# QUERY THE DATA ONCE
query_job = bigquery_client.query(query)
pred = query_job.to_dataframe()
pred.head()

In [ ]:
pred

In [ ]:
x['uu_id'] = label[x["uu_id"]]
x

In [ ]:
for col in x.columns[2:]:
    li = []
    for i in pred['uu_id']:
        li.append(x.loc[x['uu_id'] == i, col].mean())
    pred[col] = li

In [ ]:
pred

In [ ]:
pred['uu_id'] = pred['uu_id'].factorize(sort=True)[0]

In [ ]:
x_test

In [ ]:
pred = pred.apply(pd.to_numeric)

In [ ]:
predictions = gridcv_xgb.predict(pred)

In [ ]:
predictions

In [ ]:
predictions = model.predict(pred)

In [ ]:
out_df = pd.DataFrame()

In [ ]:
out_df["uu_id"] = pred["uu_id"]
out_df["total_claims"] = predictions
out_df["week_numer"] = pred["week_number"]

In [ ]:
out_df

In [ ]:
query = """
SELECT uu_id FROM `ironhacks-data.ironhacks_competition.prediction_list` 
"""

In [ ]:
# QUERY THE DATA ONCE
query_job = bigquery_client.query(query)
uu_id = query_job.to_dataframe()

In [ ]:
uuid, label = uu_id['uu_id'].factorize(sort=True)

In [ ]:
out_df["uu_id"] = label[out_df["uu_id"]]

In [ ]:
out_df

In [ ]:
out_df.to_csv("submission_prediction_output.csv", index=False)